In [1]:
import numpy as np
from functions import prepData, splitData
from crossValidation import crossValidate
from Training import TrainDTree, SplitInfo, selectBestFeature
from PredictPrune import PredictDTree, PruneDTree

In [2]:
raw = np.array([('Sunny', 'Hot', 'High', 'False', 'N'),
                ('Sunny', 'Hot', 'High', 'True', 'N'),
                ('Overcast', 'Hot', 'High', 'False', 'P'), 
                ('Rainy', 'Mild', 'High', 'False', 'P'), 
                ('Rainy', 'Cool', 'Normal', 'False', 'P'), 
                ('Rainy', 'Cool', 'Normal', 'True', 'N'), 
                ('Overcast', 'Cool', 'Normal', 'True', 'P'), 
                ('Sunny', 'Mild', 'High', 'False', 'N'), 
                ('Sunny', 'Cool', 'Normal', 'False', 'P'), 
                ('Rainy', 'Mild', 'Normal', 'False', 'P'), 
                ('Sunny', 'Mild', 'Normal', 'True', 'P'),
                ('Overcast', 'Mild', 'High', 'True', 'P'),
                ('Overcast', 'Hot', 'Normal', 'False', 'P'), 
                ('Rainy', 'Mild', 'High', 'True', 'N')],
               dtype=[('Outlook','U8'), ('Temperature','U4'), ('Humidity','U6'),
                      ('Windy','U5'), ('Class','U1')]
            )
testData = raw[['Outlook','Temperature','Humidity','Windy']]
testLabel = raw['Class']

In [3]:
SplitInfo(testData['Temperature'], testLabel)

(0.9110633930116763, 1.5566567074628228, None)

In [4]:
selectBestFeature(testData, testLabel, useRatio=False)

('Outlook', 0.24674981977443933, None, 0.24674981977443933)

In [5]:
testTree = TrainDTree(testData,testLabel)
print(testTree)

Attribute [Outlook]:
    value = Overcast, class: P
    value = Rainy, Attribute [Windy]:
        value = True, class: N
        value = False, class: P
    value = Sunny, Attribute [Humidity]:
        value = Normal, class: P
        value = High, class: N



In [6]:
carPath = './data/car.data'
carNames = ['buying','maint','doors','persons','lug_boot','safety','accept']
carData, carFeats, acceptVec = prepData(carPath, carNames, slice(-1), 'accept')

(xvData,xvLabel),xvFolds,pruningSet = splitData(carData, acceptVec)
testData,testLabel = xvData[xvFolds[0]],xvLabel[xvFolds[0]]
trainIdx = np.hstack(xvFolds[1:])
trainData,trainLabel = xvData[trainIdx],xvLabel[trainIdx]

In [7]:
z = TrainDTree(trainData, trainLabel)
z.combineChildNodes()
print(z)

Attribute [safety]:
    value = low, class: unacc
    value = med, Attribute [persons]:
        value = more, Attribute [lug_boot]:
            value = small, Attribute [buying]:
                value = low, Attribute [doors]:
                    value = 4, class: acc
                    value = 3, Attribute [maint]:
                        value = low, class: acc
                        value = vhigh, class: unacc
                        value = high, class: acc
                    value = 2, class: unacc
                    value = 5more, Attribute [maint]:
                        value = low, class: acc
                        value = vhigh, class: unacc
                        value = med, class: acc
                        value = high, class: acc
                value = vhigh, class: unacc
                value = med, Attribute [maint]:
                    value = low, class: acc
                    value = vhigh, class: unacc
                    value = med, Attribute [doors]:
 

In [8]:
PruneDTree(z, pruningSet[0], pruningSet[1])
print(z)

Attribute [safety]:
    value = low, class: unacc
    value = med, Attribute [persons]:
        value = more, Attribute [lug_boot]:
            value = small, Attribute [buying]:
                value = low, Attribute [doors]:
                    value = 4, class: acc
                    value = 3, class: acc
                    value = 2, class: unacc
                    value = 5more, Attribute [maint]:
                        value = low, class: acc
                        value = vhigh, class: unacc
                        value = med, class: acc
                        value = high, class: acc
                value = vhigh, class: unacc
                value = med, Attribute [maint]:
                    value = low, class: acc
                    value = vhigh, class: unacc
                    value = med, class: acc
                    value = high, class: unacc
                value = high, class: unacc
            value = big, Attribute [maint]:
                value = low, Att